### Create database table for Regions Clicking

In [34]:
import os, sys, psycopg2
import numpy as np
import warnings; warnings.filterwarnings("ignore")
from natsort import natsorted
from scipy import misc
from PIL import Image, ImageFilter

In [3]:
regions_dir = '../app/assets/images/2-Regions'

In [164]:
regions_subdirs = os.listdir(regions_dir)
regions_subdirs = [os.path.join(regions_dir, s, 'cyan') for s in regions_subdirs]
regions_subdirs = natsorted(regions_subdirs)

In [165]:
slices_dict = dict()
slice_shape = (621, 1406)
no_slices = 138
for i in range(0, no_slices):
    slices_dict[i] = np.ndarray(slice_shape, dtype=object)

In [178]:
def populate_slices_dict(slices_dict, regions_subdirs, start=0):
    for i, subdir in enumerate(regions_subdirs):
        try:
            images = natsorted([os.path.join(subdir, image) \
                        for image in os.listdir(subdir)])
        except:
            continue
        for sli_n, image in enumerate(images):
            im = misc.imread(image, 'L')
            for index, x in np.ndenumerate(im):
                if x:
                    matching_slis = slices_dict[sli_n][index]
                    if matching_slis == None:
                        matching_slis = list()
                    region_name = subdir.split('/')[5]
                    if region_name not in matching_slis:
                        matching_slis.append(region_name)
        print(i + start, subdir)

In [ ]:
populate_slices_dict(slices_dict, regions_subdirs[9:], 9)

(9, '../app/assets/images/2-Regions/Forebrain - Diencephalon - Dopaminergic Cluster 3 - hypothalamus/cyan')
(10, '../app/assets/images/2-Regions/Forebrain - Diencephalon - Dopaminergic Cluster 4-5 - posterior tuberculum and hypothalamus/cyan')
(11, '../app/assets/images/2-Regions/Forebrain - Diencephalon - Dopaminergic Cluster 6 - hypothalamus/cyan')
(12, '../app/assets/images/2-Regions/Forebrain - Diencephalon - Dopaminergic Cluster 7 - Caudal Hypothalamus/cyan')
(13, '../app/assets/images/2-Regions/Forebrain - Diencephalon - Eminentia Thalami/cyan')
(14, '../app/assets/images/2-Regions/Forebrain - Diencephalon - Habenula/cyan')
(15, '../app/assets/images/2-Regions/Forebrain - Diencephalon - Hypothalamus 6.7FRhcrtR-Gal4 cluster 1/cyan')
(16, '../app/assets/images/2-Regions/Forebrain - Diencephalon - Hypothalamus 6.7FRhcrtR-Gal4 cluster 2/cyan')
(17, '../app/assets/images/2-Regions/Forebrain - Diencephalon - Hypothalamus - Caudal Hypothalamus Neural Cluster/cyan')
(18, '../app/assets/i

In [176]:
print(regions_subdirs[5])

../app/assets/images/2-Regions/Forebrain - Diencephalon - Anterior pretectum cluster of vmat2 Neurons/cyan
